In [46]:
%logstop
%logstart -rtq ~/.logs/vc.py append
import seaborn as sns
sns.set()

In [19]:
from static_grader import grader

# Object-oriented exercises
## Introduction

The objective of these exercises is to develop your familiarity with Python's `class` syntax and object-oriented programming. By deepening our understanding of Python objects, we will be better prepared to work with complex data structures and machine learning models. We will develop a `Point` class capable of handling some simple linear algebra operations in 2D.

## Exercise 1: `point_repr`

The first step in defining most classes is to define their `__init__` and `__repr__` methods so that we can construct and represent distinct objects of that class. Our `Point` class should accept two arguments, `x` and `y`, and be represented by a string `'Point(x, y)'` with appropriate values for `x` and `y`.  (Note the formatting of the previous string very closely.)  

When you've written a `Point` class capable of this, execute the cell with `grader.score` for this question (do not edit that cell; you only need to modify the `Point` class).

In [44]:
class Point(object):

    def __init__(self, x, y):
        self.x = x
        self.y = y
              
    def __repr__(self):
        return f"Point({self.x}, {self.y})"


In [41]:
pt_1 = Point(2,3)

In [42]:
print(pt_1)

Point(2,3)


In [45]:
grader.score.vc__point_repr(lambda points: [str(Point(*point)) for point in points])

Your score: 1.000


**Remark:** If your score is 0, check the spacing in your `__repr__` method carefully!

## Exercise 2: add_subtract

The most basic vector operations we want our `Point` object to handle are addition and subtraction. For two points $(x_1, y_1) + (x_2, y_2) = (x_1 + x_2, y_1 + y_2)$ and similarly for subtraction. Implement a method within `Point` that allows two `Point` objects to be added together using the `+` operator, and likewise for subtraction. Once this is done, execute the `grader.score` cell for this question (do not edit that cell; you only need to modify the `Point` class.)

(Remember that `__add__` and `__sub__` methods will allow us to use the `+` and `-` operators.)

In [47]:
class Point(object):

    def __init__(self, x, y):
        self.x = x
        self.y = y
              
    def __repr__(self):
        return f"Point({self.x}, {self.y})"
    
    def __add__(self,number):
        return Point(self.x+number.x,self.y+number.y)
    def __sub__(self,number):
        return Point(self.x-number.x,self.y-number.y)

In [48]:
from functools import reduce
def add_sub_results(points):
    points = [Point(*point) for point in points]
    return [str(reduce(lambda x, y: x + y, points)), 
            str(reduce(lambda x, y: x - y, points))]

grader.score.vc__add_subtract(add_sub_results)

Your score: 1.000


## Exercise 3: multiplication

Within linear algebra there's many different kinds of multiplication: scalar multiplication, inner product, cross product, and matrix product. We're going to implement scalar multiplication and the inner product.

We can define scalar multiplication given a point $P$ and a scalar $a$ as 
$$aP=a(x,y)=(ax,ay)$$

and we can define the inner product for points $P,Q$ as
$$P\cdot Q=(x_1,y_1)\cdot (x_2, y_2) = x_1x_2 + y_1y_2$$

To test that you've implemented this correctly, compute $2(x, y) \cdot (x, y)$ for a `Point` object. Once this is done, execute the `grader.score` cell for this question (do not edit that cell; you only need to modify the `Point` class.)

(Remember that `__mul__` method will allow us to use the `*` operator. Also don't forget that the ordering of operands matters when implementing these operators.)

In [66]:
class Point(object):

    def __init__(self, x, y):
        self.x = x
        self.y = y
              
    def __repr__(self):
        return f"Point({self.x}, {self.y})"
    
    def __mul__(self,number):
        if isinstance(number,int):                        #Here we are checking that if the number that is provieded is an integer then just multiply it with two coordinate
            return Point(self.x * number,self.y * number) #We are using Point because we are creating a new object with different values of x and y
        elif isinstance(number,Point):                    #Here we are checking that if number is of type Point then take the dot product
            return self.x * number.x + self.y * number.y #We are not using the Point because we are only concerned with a single value.
        else:
            raise TypeError('Expected number to be int or Rational. Got %s' % type(number))

pt_2 =  Point(1,2) * 50


pt_2 = pt_2 * pt_2

print(pt_2)

12500


In [67]:
def mult_result(points):
    points = [Point(*point) for point in points]
    return [point*point*2 for point in points]

grader.score.vc__multiplication(mult_result)

Your score: 1.000


## Exercise 4: Distance

Another quantity we might want to compute is the distance between two points.  This is generally given for points $P_1=(x_1,y_1)$ and $P_2=(x_2,y_2)$ as 
$$D = |P_2 - P_1| = \sqrt{(x_1-x_2)^2 + (y_1-y_2)^2}.$$

Implement a method called `distance` which finds the distance from a point to another point. 

Once this is done, execute the `grader.score` cell for this question (do not edit that cell; you only need to modify the `Point` class.)

### Hint
* *You can use the `sqrt` function from the math package*.

In [4]:
from math import sqrt

class Point(object):

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def distance(self, other):
        return sqrt((self.x-other.x)**2 + (self.y-other.y)**2)
    
    def __repr__(self):
        return f"Point({self.x}, {self.y})"
    
    
p1 = Point(2,3)
p2 = Point(7,8)

    

In [5]:
def dist_result(points):
    points = [Point(*point) for point in points]
    return [points[0].distance(point) for point in points]

grader.score.vc__distance(dist_result)

Your score: 1.000


## Exercise 5: Algorithm

Now we will use these points to solve a real world problem!  We can use our Point objects to represent measurements of two different quantities (e.g. a company's stock price and volume).  One thing we might want to do with a data set is to separate the points into groups of similar points.  Here we will implement an iterative algorithm to do this which will be a specific case of the very general $k$-means clustering algorithm.  The algorithm will require us to keep track of two clusters, each of which have a list of points and a center (which is another point, not necessarily one of the points we are clustering).  After making an initial guess at the center of the two clusters, $C_1$ and $C_2$, the steps proceed as follows

1. Assign each point to $C_1$ or $C_2$ based on whether the point is closer to the center of $C_1$ or $C_2$.
2. Recalculate the center of $C_1$ and $C_2$ based on the contained points. 

See [reference](https://en.wikipedia.org/wiki/K-means_clustering#Standard_algorithm) for more information.

This algorithm will terminate in general when the assignments no longer change.  For this question, we would like you to initialize one cluster at `(1, 0)` and the other at `(-1, 0)`.  

The returned values should be the two centers of the clusters ordered by greatest `x` value.  Please return these as a list of numeric tuples $[(x_1, y_1), (x_2, y_2)]$

In order to accomplish this we will create a class called cluster which has two methods besides `__init__` which you will need to write.  The first method `update` will update the center of the Cluster given the points contained in the attribute `points`.  Remember, you after updating the center of the cluster, you will want to reassign the points and thus remove previous assignments.  The other method `add_point` will add a point to the `points` attribute.

Once this is done, execute the `grader.score` cell for this question (do not edit that cell; you only need to modify the `Cluster` class and `compute_result` function.)

In [15]:
class Cluster(object):
    def __init__(self, x, y):
        self.center = Point(x, y)   #Notice we are make self.center(i.e  is variable) equal to an object(i.e is of class Point)
        self.points = []
    
    def update(self):
        #Here we are going to do two things
        #1. First we will calculate the new center of the given points
        #2. Then we will clear out all the points contained in self.points list
        
        #For step 1 we aregong to extract out x and y values of all  the points in self.points list.
        #We are making separate lists for  that
        
        all_x = [pt.x for pt in self.points]    #Here we will first write for loop which states for all pt in self.points list, then pt.x means that extract x variable from all the pt.
        all_y = [pt.y for pt in self.points]
        
        x_c = sum(all_x)/len(all_x)            #Here we are extracting the center point for x
        y_c = sum(all_y)/len(all_y)            #Here we are extracting the center point for y
        
        self.center = Point(x_c,y_c)
        
        self.points = []
        
    def add_point(self, point):
        self.points.append(point)

In [17]:
points


NameError: name 'points' is not defined

In [41]:
def compute_result(points):
    points = [Point(*point) for point in points] # Here we are using an unpacking operator (*) which means that, Point(*point) can be written as Point(point[0],point[1]) which are x and y coordinates 
    print(points)                                # The points is bascially a list of Point objects,  we are taking in the  points in form of [[x1,y1],[x2,y2]...] and returing list of point objects
    a = Cluster(1,0)
    b = Cluster(-1,0)
    a_old = []
    for _ in range(10000): # max iterations
        #We are checking the distance of the points
        for point in points:  
            if point.distance(a.center) < point.distance(b.center):
                a.add_point(point)
            else:
                b.add_point(point)
        
        if a_old == a.points:
            break
        a_old = a.points
        a.update()
        b.update()
    
    #We are returning the value according to greatest x value
    if a.center.x > b.center.x:
        return [(a.center.x,a.center.y),(b.center.x,b.center.y)]
    else :
        return [(b.center.x,b.center.y),(a.center.x,a.center.y)]
    #return [(x, y)] * 2

In [42]:
points = [[1,2],[3,4]]


compute_result(points)

[Point(1, 2), Point(3, 4)]


ZeroDivisionError: division by zero

In [43]:
grader.score.vc__k_means(compute_result)

[Point(-107, 630), Point(-790, -305), Point(-564, -387), Point(-181, -68), Point(330, -474), Point(-295, -803), Point(407, -920), Point(-640, 20), Point(943, 177), Point(428, -391), Point(-62, -335), Point(964, -98), Point(-306, -540), Point(-103, -979), Point(393, 208), Point(-94, -689), Point(497, -273), Point(201, 903), Point(965, 416), Point(-204, -928), Point(-809, -521), Point(116, -442), Point(56, 292), Point(-1, -604), Point(-241, 54), Point(-473, -996), Point(-61, -70), Point(-496, -354), Point(443, 539), Point(-786, 905), Point(620, 581), Point(-547, 588), Point(320, 102), Point(643, 964), Point(-696, 219), Point(-449, -941), Point(685, 640), Point(-763, -178), Point(120, 638), Point(-419, -894), Point(826, -216), Point(-583, -731), Point(-909, 170), Point(848, -749), Point(156, 946), Point(595, -172), Point(436, 93), Point(561, 48), Point(535, -868), Point(-507, 424)]
Your score: 1.000


*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*